In [7]:
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path


#xml_dir = '/data/darknet/python/' + time_str + '_xml_fillter_obj/'
#image_dir = '/data/darknet/python/' + time_str + '_image_fillter_obj/'
time_str = '20190124'
image_dir = '/data/darknet/python/' + time_str + '_images_replaced_ordinary_obj_with_bkg/'
xml_dir = '/data/darknet/python/' + time_str + '_xmls_replaced_ordinary_obj_with_bkg/'



time_str = (datetime.datetime.now()).strftime("%Y%m%d")
generated_image_dir = '/data/darknet/python/' + time_str + '_images_replaced_bkg/'
os.system('mkdir -p ' + xml_dir)
os.system('mkdir -p ' + image_dir)
os.system('mkdir -p ' + generated_image_dir)
background_dir = '/data/darknet/python/background/'

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
SEED_BKG_DIR = background_dir
GENE_IMG_DIR = generated_image_dir

def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)
seed_bkg_names.sort()
seed_xml_names.sort()

for xml_name in seed_xml_names:
    xml_name_part = xml_name[-15:-4]
    print(xml_name_part)
    if not (int(xml_name_part) >= 460000):
        continue
    background_img_data = cv2.imread(SEED_BKG_DIR + seed_bkg_names[10])
    print(background_img_data.shape)
    
    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    bboxes = get_bboxes_from_etree(tree)
    print(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    past_to_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', bboxes, background_img_data)

00000460500
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174021.565039_00000460500.jpg
00000460501
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174021.770322_00000460501.jpg
00000460502
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174021.976159_00000460502.jpg
00000460503
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174022.185984_00000460503.jpg
00000460509
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174023.431675_00000460509.jpg
00000460510
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174023.638927_00000460510.jpg
00000460511
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174023.837314_00000460511.jpg
00000460515
(1920, 2560, 3)
/data/darknet/python/20190124_imag

00000460640
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174050.496029_00000460640.jpg
00000460641
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174050.695296_00000460641.jpg
00000460645
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174051.493308_00000460645.jpg
00000460648
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174052.107931_00000460648.jpg
00000460660
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174054.542865_00000460660.jpg
00000460664
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174055.374592_00000460664.jpg
00000460665
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174055.585162_00000460665.jpg
00000460666
(1920, 2560, 3)
/data/darknet/python/20190124_imag

00000460741
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174111.656983_00000460741.jpg
00000460743
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174112.079457_00000460743.jpg
00000460746
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174112.704363_00000460746.jpg
00000460752
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174113.986069_00000460752.jpg
00000460770
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174117.695355_00000460770.jpg
00000460772
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174118.126351_00000460772.jpg
00000460777
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174119.178888_00000460777.jpg
00000460781
(1920, 2560, 3)
/data/darknet/python/20190124_imag

00000460918
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174148.611451_00000460918.jpg
00000460919
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174148.816360_00000460919.jpg
00000460923
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174149.674875_00000460923.jpg
00000460924
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174149.888055_00000460924.jpg
00000460925
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174150.095709_00000460925.jpg
00000460926
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174150.314084_00000460926.jpg
00000460927
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174150.530110_00000460927.jpg
00000460928
(1920, 2560, 3)
/data/darknet/python/20190124_imag

00000460987
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174203.218070_00000460987.jpg
00000460988
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174203.436439_00000460988.jpg
00000460989
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174203.640192_00000460989.jpg
00000460991
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174204.054128_00000460991.jpg
00000460992
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174204.273021_00000460992.jpg
00000460993
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174204.480248_00000460993.jpg
00000460994
(1920, 2560, 3)
/data/darknet/python/20190124_images_replaced_ordinary_obj_with_bkg/20190124174204.678769_00000460994.jpg
00000460995
(1920, 2560, 3)
/data/darknet/python/20190124_imag